In [1]:
#import all the necessary packages.
!pip install plotly
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

In [3]:
# we have give a json file which consists of all information about
# the products
# loading the data using pandas' read_json file.
import os
data = pd.read_json('tops_fashion.json')

In [5]:
print ('Number of data points : ', data.shape[0], \
       '\nNumber of features/variables:', data.shape[1])

Number of data points :  183138 
Number of features/variables: 19


In [6]:
# We have total 72 unique type of product_type_names
print(data['product_type_name'].describe())

# 91.62% (167794/183138) of the products are shirts,

count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object


In [7]:
# names of different product types
print(data['product_type_name'].unique())

['SHIRT' 'SWEATER' 'APPAREL' 'OUTDOOR_RECREATION_PRODUCT'
 'BOOKS_1973_AND_LATER' 'PANTS' 'HAT' 'SPORTING_GOODS' 'DRESS' 'UNDERWEAR'
 'SKIRT' 'OUTERWEAR' 'BRA' 'ACCESSORY' 'ART_SUPPLIES' 'SLEEPWEAR'
 'ORCA_SHIRT' 'HANDBAG' 'PET_SUPPLIES' 'SHOES' 'KITCHEN' 'ADULT_COSTUME'
 'HOME_BED_AND_BATH' 'MISC_OTHER' 'BLAZER' 'HEALTH_PERSONAL_CARE'
 'TOYS_AND_GAMES' 'SWIMWEAR' 'CONSUMER_ELECTRONICS' 'SHORTS' 'HOME'
 'AUTO_PART' 'OFFICE_PRODUCTS' 'ETHNIC_WEAR' 'BEAUTY'
 'INSTRUMENT_PARTS_AND_ACCESSORIES' 'POWERSPORTS_PROTECTIVE_GEAR' 'SHIRTS'
 'ABIS_APPAREL' 'AUTO_ACCESSORY' 'NONAPPARELMISC' 'TOOLS' 'BABY_PRODUCT'
 'SOCKSHOSIERY' 'POWERSPORTS_RIDING_SHIRT' 'EYEWEAR' 'SUIT'
 'OUTDOOR_LIVING' 'POWERSPORTS_RIDING_JACKET' 'HARDWARE' 'SAFETY_SUPPLY'
 'ABIS_DVD' 'VIDEO_DVD' 'GOLF_CLUB' 'MUSIC_POPULAR_VINYL'
 'HOME_FURNITURE_AND_DECOR' 'TABLET_COMPUTER' 'GUILD_ACCESSORIES'
 'ABIS_SPORTS' 'ART_AND_CRAFT_SUPPLY' 'BAG' 'MECHANICAL_COMPONENTS'
 'SOUND_AND_RECORDING_EQUIPMENT' 'COMPUTER_COMPONENT' 'JEWELRY'
 'B

In [8]:
# find the 10 most frequent product_type_names.
product_type_count = Counter(list(data['product_type_name']))
product_type_count.most_common(10)

[('SHIRT', 167794),
 ('APPAREL', 3549),
 ('BOOKS_1973_AND_LATER', 3336),
 ('DRESS', 1584),
 ('SPORTING_GOODS', 1281),
 ('SWEATER', 837),
 ('OUTERWEAR', 796),
 ('OUTDOOR_RECREATION_PRODUCT', 729),
 ('ACCESSORY', 636),
 ('UNDERWEAR', 425)]

In [9]:
# there are 10577 unique brands
print(data['brand'].describe())

# 183138 - 182987 = 151 missing values.

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object


In [10]:
brand_count = Counter(list(data['brand']))
brand_count.most_common(10)

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

In [11]:

print(data['color'].describe())


# we have 7380 unique colors
# 7.2% of products are black in color
# 64956 of 183138 products have brand information. That's approx 35.4%.

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object


In [12]:
color_count = Counter(list(data['color']))
color_count.most_common(10)

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

In [13]:
 
print(data['formatted_price'].describe())

# Only 28,395 (15.5% of whole data) products with price information

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object


In [14]:
price_count = Counter(list(data['formatted_price']))
price_count.most_common(10)

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

In [15]:
print(data['title'].describe())

# All of the products have a title. 
# Titles are fairly descriptive of what the product is. 
# We use titles extensively in this workshop 
# as they are short and informative.


count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object


In [17]:
data.to_pickle('180k_apparel_data')

In [18]:
# consider products which have price information
# data['formatted_price'].isnull() => gives the information 
#about the dataframe row's which have null values price == None|Null
data = data.loc[~data['formatted_price'].isnull()]
print('Number of data points After eliminating price=NULL :', data.shape[0])

Number of data points After eliminating price=NULL : 28395


In [19]:
# consider products which have color information
# data['color'].isnull() => gives the information about the dataframe row's which have null values price == None|Null
data =data.loc[~data['color'].isnull()]
print('Number of data points After eliminating color=NULL :', data.shape[0])

Number of data points After eliminating color=NULL : 28385


In [21]:
data.to_pickle('28k_apparel_data')

In [22]:
# read data from pickle file from previous stage
data = pd.read_pickle('28k_apparel_data')

# find number of products that have duplicate titles.
print(sum(data.duplicated('title')))
# we have 2325 products which have same title but different color


2325


In [23]:
# read data from pickle file from previous stage
data = pd.read_pickle('28k_apparel_data')

In [24]:
data.head()

asin author                         availability availability_type  \
4   B004GSI2OS   None  Usually ships in 6-10 business days               now   
6   B012YX2ZPI   None   Usually ships in 4-5 business days               now   
11  B001LOUGE4   None                                 None              None   
15  B003BSRPB0   None  Usually ships in 6-10 business days               now   
21  B014ICEDNA   None   Usually ships in 4-5 business days               now   

                          brand              color editorial_reivew  \
4                   FeatherLite  Onyx Black/ Stone              NaN   
6   HX-Kingdom Fashion T-shirts              White              NaN   
11                 Fitness Etc.              Black              NaN   
15                  FeatherLite              White              NaN   
21                        FNC7C             Purple              NaN   

                                     editorial_review formatted_price  \
4                                                              $26.26   
6   This Personalized Special Olympics World Games...           $9.99   
11  Light Weight 2x1 Boy Beater Tank Top.  Great t...          $11.99   
15  FeatherLite Ladies' Moisture Free Mesh Sport S...          $20.54   
21  Supernatural Chibis Sam Dean And Castiel Women...           $7.50   

                                      large_image_url manufacturer  \
4   https://images-na.ssl-images-amazon.com/images...         None   
6   https://images-na.ssl-images-amazon.com/images...         None   
11  https://images-na.ssl-images-amazon.com/images...         None   
15  https://images-na.ssl-images-amazon.com/images...         None   
21  https://images-na.ssl-images-amazon.com/images...         None   

                                     medium_image_url model product_type_name  \
4   https://images-na.ssl-images-amazon.com/images...  None             SHIRT   
6   https://images-na.ssl-images-amazon.com/images...  None             SHIRT   
11  https://images-na.ssl-images-amazon.com/images...  None             SHIRT   
15  https://images-na.ssl-images-amazon.com/images...  None             SHIRT   
21  https://images-na.ssl-images-amazon.com/images...  None             SHIRT   

   publisher                                            reviews  \
4       None  [False, https://www.amazon.com/reviews/iframe?...   
6       None  [False, https://www.amazon.com/reviews/iframe?...   
11      None  [False, https://www.amazon.com/reviews/iframe?...   
15      None  [False, https://www.amazon.com/reviews/iframe?...   
21      None  [True, https://www.amazon.com/reviews/iframe?a...   

                     sku                                    small_image_url  \
4                   None  https://images-na.ssl-images-amazon.com/images...   
6                   None  https://images-na.ssl-images-amazon.com/images...   
11                  None  https://images-na.ssl-images-amazon.com/images...   
15  HT-2001_Lime-1149-XL  https://images-na.ssl-images-amazon.com/images...   
21                  None  https://images-na.ssl-images-amazon.com/images...   

                                                title  
4   Featherlite Ladies' Long Sleeve Stain Resistan...  
6   Women's Unique 100% Cotton T - Special Olympic...  
11             Ladies Cotton Tank 2x1 Ribbed Tank Top  
15  FeatherLite Ladies' Moisture Free Mesh Sport S...  
21  Supernatural Chibis Sam Dean And Castiel Short...

In [25]:
# Remove All products with very few words in title
data_sorted = data[data['title'].apply(lambda x: len(x.split())>4)]
print("After removal of products with short description:", data_sorted.shape[0])

After removal of products with short description: 27949


In [26]:
# Sort the whole data based on title (alphabetical order of title) 
data_sorted.sort_values('title',inplace=True, ascending=False)
data_sorted.head()

asin author                        availability  \
61973   B06Y1KZ2WB   None  Usually ships in 1-2 business days   
133820  B010RV33VE   None  Usually ships in 4-5 business days   
81461   B01DDSDLNS   None  Usually ships in 4-5 business days   
75995   B00X5LYO9Y   None  Usually ships in 4-5 business days   
151570  B00WPJG35K   None  Usually ships in 4-5 business days   

       availability_type     brand        color  \
61973                now    Éclair   Black/Pink   
133820               now  xiaoming         Pink   
81461                now  xiaoming        White   
75995                now  xiaoming  Red Anchors   
151570               now  xiaoming        White   

                                         editorial_reivew editorial_review  \
61973   Perfect for summer adventures, feminine touch ...              NaN   
133820  <p>Korean Style Womens' Sleeveless Loose Long ...              NaN   
81461   <br>1.xiaoming is a modern lifestyle brand tha...              NaN   
75995   Sleeve Style: Short Sleeve<br />Patten:Stripes...              NaN   
151570  Material :Lace<br />This cardigan is imported ...              NaN   

       formatted_price                                    large_image_url  \
61973           $24.99  https://images-na.ssl-images-amazon.com/images...   
133820          $18.19  https://images-na.ssl-images-amazon.com/images...   
81461           $21.58  https://images-na.ssl-images-amazon.com/images...   
75995           $15.91  https://images-na.ssl-images-amazon.com/images...   
151570          $14.32  https://images-na.ssl-images-amazon.com/images...   

       manufacturer                                   medium_image_url  \
61973          None  https://images-na.ssl-images-amazon.com/images...   
133820         None  https://images-na.ssl-images-amazon.com/images...   
81461          None  https://images-na.ssl-images-amazon.com/images...   
75995          None  https://images-na.ssl-images-amazon.com/images...   
151570         None  https://images-na.ssl-images-amazon.com/images...   

                 model product_type_name publisher  \
61973             None             SHIRT      None   
133820  xiaona10326758             SHIRT      None   
81461       MxiaoCotds             SHIRT      None   
75995             None             SHIRT      None   
151570            None             SHIRT      None   

                                                  reviews   sku  \
61973   [False, https://www.amazon.com/reviews/iframe?...  None   
133820  [False, https://www.amazon.com/reviews/iframe?...  None   
81461   [False, https://www.amazon.com/reviews/iframe?...  None   
75995   [False, https://www.amazon.com/reviews/iframe?...  None   
151570  [False, https://www.amazon.com/reviews/iframe?...  None   

                                          small_image_url  \
61973   https://images-na.ssl-images-amazon.com/images...   
133820  https://images-na.ssl-images-amazon.com/images...   
81461   https://images-na.ssl-images-amazon.com/images...   
75995   https://images-na.ssl-images-amazon.com/images...   
151570  https://images-na.ssl-images-amazon.com/images...   

                                                    title  
61973   Éclair Women's Printed Thin Strap Blouse Black...  
133820  xiaoming Womens Sleeveless Loose Long T-shirts...  
81461   xiaoming Women's White Long Sleeve Single Brea...  
75995   xiaoming Stripes Tank Patch/Bear Sleeve Anchor...  
151570  xiaoming Sleeve Sheer Loose Tassel Kimono Woma...

In [27]:
indices = []
for i,row in data_sorted.iterrows():
    indices.append(i)

In [28]:
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
    
    previous_i = i

    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = data['title'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:

        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = data['title'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))

        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [29]:
data = data.loc[data['asin'].isin(stage1_dedupe_asins)]

In [30]:
print('Number of data points : ', data.shape[0])

Number of data points :  17593


In [31]:
data.to_pickle('17k_apperal_data')

In [32]:
data = pd.read_pickle('17k_apperal_data')

In [ ]:
# This code snippet takes significant amount of time.
# O(n^2) time.
# Takes about an hour to run on a decent computer.

indices = []
for i,row in data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(data['asin'].loc[i])
    # consider the first apperal's title
    a = data['title'].loc[i].split()
    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    for j in indices:
        
        b = data['title'].loc[j].split()
        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
        
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        # if the number of words in which both strings differ are < 3 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)